10. Partition news data by major topic. Use the appropriate model for the topic and predict. Only keep: news_source, date, major_topic, slant_prediction (this is what our model does) --- in fact, it would be great if we could append slant score (prob. republican) to the major topic prediction


In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
import re
import textwrap

def insert_chars_split_marker(text, cc=2500):
    # FIXME: text still has number
    text = re.sub('\d+', '', text)
    out = '|'.join(textwrap.wrap(text, cc))
    return out

def insert_words_split_marker(text, wc=500):
    text = re.sub('\d+', '', text)
    words = text.split()
    out = ''
    for i, w in enumerate(words):
        if i != 0 and i % wc == 0:
            out += '|' + w
        else:
            out += ' ' + w
    return out

In [3]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfTransformer

## Party prediction

In [4]:
for t in [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]:
    vect_name = '../new_models/major_%s_vec_count_cong_23gram.joblib' % t
    clf_name = '../new_models/major_%s_cong_clf_elastic.joblib' % t

    vect = joblib.load(vect_name)
    clf = joblib.load(clf_name)
    print t, len(vect.vocabulary_)
    
    total = 0
    for df in pd.read_csv('../data//major_%s_news_predict_dedup.csv' % t, iterator=True, chunksize=1000):
        if False:
            df['text'] = df['text'].apply(lambda c: insert_chars_split_marker(c, 2500))
            s = df['text'].str.split('|', expand=True).stack()
            i = s.index.get_level_values(0)
            new_df = df.loc[i].copy()
            new_df['chunk'] = s.index.get_level_values(1)
            new_df['text'] = s.values
            df = new_df.reset_index(drop=True)
        X = vect.transform(df.text.astype(str))
        tfidf = TfidfTransformer()
        X = tfidf.fit_transform(X)
        y_pred = clf.predict(X)
        df['slant_prediction'] = y_pred
        df['slant_score'] = clf.predict_proba(X)[:,1]
        if total == 0:
            with open("../new_data/major_%d_news_predict_party.csv" % t, "w") as fp:
                df[['date', 'news_source', 'topic', 'slant_prediction', 'slant_score']].to_csv(fp, index=False)
        else:
            with open("../new_data/major_%d_news_predict_party.csv" % t, "a") as fp:
                df[['date', 'news_source', 'topic', 'slant_prediction', 'slant_score']].to_csv(fp, header=False, index=False)
        total += 1
        print total

3 14459
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
